In [3]:
!pip install TESpy

In [4]:
#################################################################
#DATA LOADING
#################################################################
from google.colab import files
uploaded = files.upload()

Saving HP_case_data.xlsx to HP_case_data.xlsx


In [5]:
import pandas as pd
file_path = "HP_case_data.xlsx"
source_df = pd.read_excel(file_path, sheet_name="Heat source")
sink_df   = pd.read_excel(file_path, sheet_name="Heat sink")


/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


In [33]:
# === Convert times ===
source_df['start measurement'] = pd.to_datetime(source_df['start measurement'])
source_df['end measurement'] = pd.to_datetime(source_df['end measurement'])

sink_df['start measurement']   = pd.to_datetime(sink_df['start measurement'])
sink_df['end measurement']   = pd.to_datetime(sink_df['end measurement'])

# === Extract columns as numpy arrays ===
Temp_in_source  = source_df['T_in[degC'].to_numpy()
Temp_out_source = source_df['T_out[degC]'].to_numpy()
Start_source = source_df['start measurement'].to_numpy()
End_source   = source_df['end measurement'].to_numpy()

Temp_in_sink  = sink_df['T_in[degC'].to_numpy()
Temp_out_sink = sink_df['T_out[degC]'].to_numpy()
Energy_sink = sink_df['Energy[kWh]'].to_numpy()
Start_sink   = sink_df['start measurement'].to_numpy()
End_sink   = sink_df['end measurement'].to_numpy()

Energy_sink = Energy_sink*1000
Temp_in_source

array([329.30355761, 329.2727304 , 329.40964915, 329.92624347,
       330.63127795, 331.25616999, 330.37447521, 329.56320546,
       329.28582663, 329.5291198 ])

In [34]:
from tespy.networks import Network
from tespy.components import Compressor, Valve, CycleCloser, Source, Sink, HeatExchanger, Condenser
from tespy.connections import Connection
import matplotlib.pyplot as plt

In [36]:
####FINAL######
class HeatPumpModel:

    def __init__(self, refrigerant='R1233zd(E)', eff_compressor=0.85):
        self.refrigerant = refrigerant
        self.eff_compressor = eff_compressor
        self._build_network()


    def _build_network(self):
        fluids = [self.refrigerant,"water"]
        self.nw = Network(fluids=fluids, iterinfo=False)

# Components
        self.cc = CycleCloser('Cycle closer')
        self.comp = Compressor('Compressor')
        self.cond = Condenser('Condenser')
        self.valve = Valve('Expansion valve')
        self.evap = HeatExchanger('Evaporator')
        self.Evap_water_in = Source('Evap_water_in')
        self.Evap_water_out = Sink('Evap_water_out')
        self.Condenser_water_in = Source('Condenser_water_in')
        self.Condenser_water_out = Sink('Condenser_water_out')


# Connections (refrigerant loop)
        c1 = Connection(self.cc, 'out1', self.evap, 'in1', label='evap_inlet')
        c0 = Connection(self.valve, 'out1', self.cc, 'in1', label='valve_out')
        c2 = Connection(self.evap, 'out1', self.comp, 'in1', label='suction')
        c3 = Connection(self.comp, 'out1', self.cond, 'in1', label='discharge')
        c4 = Connection(self.cond, 'out1', self.valve, 'in1', label='valve_in')


# Connections (water loop)
        c5 = Connection(self.Condenser_water_in, 'out1', self.cond, 'in2', label='condensor_water_in')
        c6 = Connection(self.cond, 'out2', self.Condenser_water_out, 'in1', label='condensor_water_outlet')
        c7 = Connection(self.Evap_water_in, 'out1', self.evap, 'in2', label='evaporator_water_inlet')
        c8 = Connection(self.evap, 'out2', self.Evap_water_out, 'in1', label='evaporator_water_outlet')

        self.nw.add_conns(c0, c1, c2, c3, c4, c5, c6, c7, c8)

        # Store references for easy access
        self.c0, self.c1, self.c2, self.c3, self.c4, self.c5, self.c6, self.c7, self.c8 = c0, c1, c2, c3, c4, c5, c6, c7, c8


    def set_design_conditions(self):
        # --- Define design conditions ---
        self.c2.set_attr(x=1, T=313-5, fluid={self.refrigerant: 1})  # evaporator outlet (vapor)
        self.c4.set_attr(T=363+10)  # condenser outlet temp (~90°C)

        self.cond.set_attr(Q=-1012e3)

        self.comp.set_attr(eta_s=self.eff_compressor)
        self.cond.set_attr(pr1=0.99, pr2=0.99)
        self.evap.set_attr(pr1=0.99, pr2=0.99)

        self.c5.set_attr(p=2e5, T=313, fluid={"water": 1})
        self.c6.set_attr(T=363)

        self.c7.set_attr(p=2e5, T=313, fluid={ "water": 1})
        self.c8.set_attr(T=283)



    def solve_design(self):
        self.set_design_conditions()
        self.nw.solve('design')
        self.nw.save('design')
        # ✅ save design point for off-design

        self.nw.print_results()
        self.nw.results["Connection"]
        self._extract_performance()
        print("✅ Design point solved successfully.")
        print(f"COP = {self.COP:.2f}, Compressor Power = {self.W_comp/1e3:.1f} kW")
        print("\n--- Design Point ---")
        print(f"Q_cond = {self.Q_cond/1e3:.1f} kW")
        print(f"Q_evap = {self.Q_evap/1e3:.1f} kW")
        print(f"W_comp = {self.W_comp/1e3:.1f} kW")
        print(f"COP = {self.COP:.2f}")

    def _extract_performance(self):
        self.Q_cond = abs(self.cond.Q.val)
        self.Q_evap = abs(self.evap.Q.val)
        self.W_comp = abs(self.comp.P.val)
        self.COP = self.Q_cond / self.W_comp

    def solve_offdesign(self, Q_cond=None, T_evap=None, eff_compressor=None, source_in=None, source_out=None, sink_in=None, sink_out=None):
        """Solve the model for off-design conditions."""
        print("\n--- Solving Off-Design Case ---")

        # Modify inlet temperatures or efficiency if provided
        if T_evap:
            self.c2.set_attr(T=T_evap)  # evaporator temp
        if Q_cond:
            self.cond.set_attr(Q=-Q_cond)
        if eff_compressor:
            self.comp.set_attr(eta_s=eff_compressor)
        if source_in:
            self.c7.set_attr(T=source_in)
        if source_out:
            self.c8.set_attr(T=source_out)
        if sink_in:
            self.c5.set_attr(T=sink_in)
        if sink_out:
            self.c6.set_attr(T=sink_out)

        # Set mode to 'offdesign' and specify which components are "design" referenced
        self.nw.solve('offdesign', design_path='design')

        # Extract and print performance
        self.nw.print_results()
        self.nw.results["Connection"]
        self._extract_performance()
        print("✅ Off-design point solved successfully.")
        print(f"COP = {self.COP:.2f}, Compressor Power = {self.W_comp/1e3:.1f} kW")
        print("\n--- Off-Design Point ---")
        print(f"Q_cond = {self.Q_cond/1e3:.1f} kW")
        print(f"Q_evap = {self.Q_evap/1e3:.1f} kW")
        print(f"W_comp = {self.W_comp/1e3:.1f} kW")
        print(f"COP = {self.COP:.2f}")



# 🔁 Parametric Off-design Analysis
    def run_parametric(self, Q_cond=None, T_evap=None, eff_compressor=None, source_in=None, source_out=None, sink_in=None, sink_out=None):
        results = []
        print("\n--- Running Off-Design Parametric Simulation ---")

        for T_evap in Temp_in_source:

            for Q_cond in Energy_sink:
                try:
                    self.solve_offdesign(Q_cond=Q_cond, T_evap=T_evap-5, eff_compressor=eff_compressor, source_in=source_in, source_out=source_out, sink_in=sink_in, sink_out=sink_out)
                    results.append({
                        "Q_cond": Q_cond,
                        "COP": self.COP,
                        "W_comp [kW]": self.W_comp / 1e3,
                        "Q_cond [kW]": self.Q_cond / 1e3,
                        "Q_evap [kW]": self.Q_evap / 1e3,
                    })
                    print(f"Q_cond={Q_cond} K → COP={self.COP:.2f}")
                except Exception as e:
                    print(f"⚠️ Failed at T_evap={T_evap}: {e}")

        # Convert results to DataFrame
        df = pd.DataFrame(results)
        self.results_df = df

        return df


hp = HeatPumpModel()
hp.solve_design()
hp.solve_offdesign(source_in=300)
hp.solve_offdesign(T_evap=313-10)
hp.solve_offdesign(Q_cond=1700e3)
hp.solve_offdesign(eff_compressor=0.8)
hp.run_parametric(Q_cond= Energy_sink, T_evap= T_evap, eff_compressor=0.85, source_out=293, sink_in=333, sink_out=363)


DEBUG:TESPyLogger:Default unit specifications:
temperature: kelvin
temperature_difference: delta_degC
enthalpy: J/kg
specific_energy: J/kg
entropy: J/kg/K
pressure: Pa
mass_flow: kg/s
volumetric_flow: m3/s
specific_volume: m3/kg
power: W
heat: W
quality: 1
vapor_mass_fraction: 1
efficiency: 1
ratio: 1
length: m
speed: m/s
area: m2
thermal_conductivity: W/m/K
heat_transfer_coefficient: W/K
None: 1
DEBUG:TESPyLogger:Default mass_flow limits
min: -1000000000000.0 1 kilogram / second
max: 1000000000000.0 1 kilogram / second
DEBUG:TESPyLogger:Default pressure limits
min: 200.0 1 pascal
max: 30000000.0 1 pascal
DEBUG:TESPyLogger:Default enthalpy limits
min: 1000.0 1 joule / kilogram
max: 7000000.0 1 joule / kilogram
DEBUG:TESPyLogger:Created connection from Cycle closer (out1) to Evaporator (in1).
DEBUG:TESPyLogger:Created connection from Expansion valve (out1) to Cycle closer (in1).
DEBUG:TESPyLogger:Created connection from Evaporator (out1) to Compressor (in1).
DEBUG:TESPyLogger:Created co


##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 2.75e+05 | 5.76e+00 | -8.67e+05 | 8.50e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q |       kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | -1.01e+06 | 3.57e+04 | 2.84e+01 | 1.05e+01 | 6.00e+01 |  1.05e+01

INFO:TESPyLogger:Determined h by known p and T at condensor_water_in.
INFO:TESPyLogger:Determined h by known p and T at condensor_water_outlet.
INFO:TESPyLogger:Determined h by known p and T at evaporator_water_inlet.
INFO:TESPyLogger:Determined h by known p and T at evaporator_water_outlet.
DEBUG:TESPyLogger:Generic fluid property specification complete.
INFO:TESPyLogger:Network initialised.
INFO:TESPyLogger:Starting solver.
DEBUG:TESPyLogger:Number of connection equations: 1.
DEBUG:TESPyLogger:Number of bus equations: 0.
DEBUG:TESPyLogger:Number of component equations: 5.
DEBUG:TESPyLogger:Number of user defined equations: 0.
DEBUG:TESPyLogger:Total number of variables: 6.
INFO:TESPyLogger:Postprocessing complete.
INFO:TESPyLogger:Calculation complete.
RESULT:TESPyLogger:
##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| 


##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 3.01e+05 | 6.82e+00 | -8.95e+05 | 8.50e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q |       kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | -1.01e+06 | 3.57e+04 | 2.84e+01 | 1.05e+01 | 6.00e+01 |  1.05e+01

INFO:TESPyLogger:Network initialised.
INFO:TESPyLogger:Starting solver.
DEBUG:TESPyLogger:Number of connection equations: 1.
DEBUG:TESPyLogger:Number of bus equations: 0.
DEBUG:TESPyLogger:Number of component equations: 5.
DEBUG:TESPyLogger:Number of user defined equations: 0.
DEBUG:TESPyLogger:Total number of variables: 6.
INFO:TESPyLogger:Postprocessing complete.
INFO:TESPyLogger:Calculation complete.
RESULT:TESPyLogger:
##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 5.27e+05 | 6.82e+00 | -8.95e+05 | 8.00e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q

...

DEBUG:TESPyLogger:Switched components from design to offdesign.
DEBUG:TESPyLogger:Updated fluid property SI values and fluid mass fraction for user specified connection parameters.
INFO:TESPyLogger:Determined h and p by known T and x at suction.
INFO:TESPyLogger:Determined h by known p and T at condensor_water_in.
INFO:TESPyLogger:Determined h by known p and T at condensor_water_outlet.
INFO:TESPyLogger:Determined h by known p and T at evaporator_water_inlet.
INFO:TESPyLogger:Determined h by known p and T at evaporator_water_outlet.
DEBUG:TESPyLogger:Generic fluid property specification complete.
INFO:TESPyLogger:Network initialised.
INFO:TESPyLogger:Starting solver.
DEBUG:TESPyLogger:Number of connection equations: 1.
DEBUG:TESPyLogger:Number of bus equations: 0.
DEBUG:TESPyLogger:Number of component equations: 5.
DEBUG:TESPyLogger:Number of user defined equations: 0.
DEBUG:TESPyLogger:Total number of variables: 6.
INFO:TESPyLogger:Postprocessing complete.
INFO:TESPyLogger:Calculation


##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 9.63e+10 | 3.43e+00 | -7.44e+05 | 8.50e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q |       kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | -5.00e+11 | 2.27e+10 | 2.20e+01 | 1.05e+01 | 4.00e+01 |  1.05e+01

INFO:TESPyLogger:Determined h by known p and T at condensor_water_outlet.
INFO:TESPyLogger:Determined h by known p and T at evaporator_water_inlet.
INFO:TESPyLogger:Determined h by known p and T at evaporator_water_outlet.
DEBUG:TESPyLogger:Generic fluid property specification complete.
INFO:TESPyLogger:Network initialised.
INFO:TESPyLogger:Starting solver.
DEBUG:TESPyLogger:Number of connection equations: 1.
DEBUG:TESPyLogger:Number of bus equations: 0.
DEBUG:TESPyLogger:Number of component equations: 5.
DEBUG:TESPyLogger:Number of user defined equations: 0.
DEBUG:TESPyLogger:Total number of variables: 6.
INFO:TESPyLogger:Postprocessing complete.
INFO:TESPyLogger:Calculation complete.
RESULT:TESPyLogger:
##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 9.63e+10 | 3.43e+00 | -7.44e+05 | 8.50e-01 |
+-----------


##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 9.63e+10 | 3.43e+00 | -7.44e+05 | 8.50e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q |       kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | -5.00e+11 | 2.27e+10 | 2.20e+01 | 1.05e+01 | 4.00e+01 |  1.05e+01

,Q_cond,COP,W_comp [kW],Q_cond [kW],Q_evap [kW]
0,9.994911e+05,5.165293,1.935013e+02,9.994911e+02,8.059897e+02
1,9.970550e+05,5.165293,1.930297e+02,9.970550e+02,8.040252e+02
2,1.003651e+06,5.165293,1.943067e+02,1.003651e+03,8.093445e+02
3,1.009563e+06,5.165293,1.954514e+02,1.009563e+03,8.141121e+02
4,1.011221e+06,5.165293,1.957722e+02,1.011221e+03,8.154486e+02
...,...,...,...,...,...
135,1.007731e+06,5.193013,1.940552e+02,1.007731e+03,8.136758e+02
136,NaN,5.193013,9.628322e+07,5.000000e+08,4.037168e+08
137,NaN,5.193013,9.628322e+07,5.000000e+08,4.037168e+08
138,NaN,5.193013,9.628322e+07,5.000000e+08,4.037168e+08
